## Кейс музыкальный дистрибьютер

Дана датабаза chinook.db, в ней информация по музыке, покупкам и клиентам. Необходимо ответить на ряд бизнес-вопросов

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
select name,type
from sqlite_master
where type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Выбор артиста для подписания на лейбл
Получить данные по продажам каждого жанра в США, процентное отношение и количественное. А так же какого жанра артиста следует подписать лейблу Hip-Hop, Punk, Pop, Blues.

In [3]:
%%sql
select t.track_id,
    t.name track_name,
    g.name genre_name
from track t
join genre g
on g.genre_id=t.genre_id
limit 10;

 * sqlite:///chinook.db
Done.


track_id,track_name,genre_name
1,For Those About To Rock (We Salute You),Rock
2,Balls to the Wall,Rock
3,Fast As a Shark,Rock
4,Restless and Wild,Rock
5,Princess of the Dawn,Rock
6,Put The Finger On You,Rock
7,Let's Get It Up,Rock
8,Inject The Venom,Rock
9,Snowballed,Rock
10,Evil Walks,Rock


In [4]:
%%sql
select c.customer_id,
    c.first_name||' '||c.last_name name,
    c.country,
    il.track_id
from invoice i
left join customer c
on c.customer_id=i.customer_id
left join invoice_line il
on il.invoice_id=i.invoice_id
where c.country='USA'
limit 10;

 * sqlite:///chinook.db
Done.


customer_id,name,country,track_id
16,Frank Harris,USA,1435
16,Frank Harris,USA,1761
16,Frank Harris,USA,3377
16,Frank Harris,USA,1785
16,Frank Harris,USA,442
16,Frank Harris,USA,1951
16,Frank Harris,USA,2334
16,Frank Harris,USA,22
16,Frank Harris,USA,1488
16,Frank Harris,USA,1977


In [5]:
%%sql
with purchases as (
    select c.customer_id,
        c.first_name||' '||c.last_name name,
        c.country,
        il.track_id
    from invoice i
    left join customer c
    on c.customer_id=i.customer_id
    left join invoice_line il
    on il.invoice_id=i.invoice_id
    where c.country='USA'),

track_genre as (
    select t.track_id,
        t.name track_name,
        g.name genre_name
    from track t
    join genre g
    on g.genre_id=t.genre_id)

select tg.genre_name,
    count(tg.track_name) as purchases,
    ROUND((cast(count(tg.track_name) AS FLOAT) / (SELECT COUNT(*) from purchases)),2) percentage_sold
from purchases p
left join track_genre tg
on tg.track_id=p.track_id
group by 1
order by 2 desc;

 * sqlite:///chinook.db
Done.


genre_name,purchases,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


Основываясь на продажах по жанрам в Америке, следует подписать артиста жанра панк, т.к артисты этого жанра более продаваемы. А  так же следует заметить, что продажи в жанре рок составляют 53% от общего кол-ва продаж, так что было бы не плохо подписать еще и рокеров.

## Анализ продаж агентов

Нужно найти общую стоимость продаж для каждого агента по продажам, дополнить релевантной статистикой.

In [6]:
%%sql
select * from employee;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [7]:
%%sql

with customer_support_rep_sales as (
     select i.customer_id,
         c.support_rep_id,
         sum(i.total) total
     from invoice i
     join customer c on i.customer_id = c.customer_id
     group by 1,2)

select e.first_name || " " || e.last_name employee,
    date(e.hire_date) hire_date,
    round(sum(csrs.total),2) total_sales
from customer_support_rep_sales csrs
join employee e on e.employee_id = csrs.support_rep_id
group by 1;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01,1731.51
Margaret Park,2017-05-03,1584.0
Steve Johnson,2017-10-17,1393.92


Несмотря на то, что у лучшего агента и худшего разница в продажах составляет 20%, это обуславливается их временем работы (Стив пришел позже).

## Анализ продаж по странам

In [8]:
%%sql

with country_or_other as (
    select
        case
            when (select count(*) from customer where country = c.country) = 1 then "Other"
            else c.country
        end as country,
        c.customer_id,
        il.*
    from invoice_line il
    join invoice i on i.invoice_id = il.invoice_id
    join customer c on c.customer_id = i.customer_id)

select
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
from
    (select country,
        count(distinct customer_id) customers,
        round(SUM(unit_price),2) total_sales,
        round((SUM(unit_price) / count(distinct customer_id)),2) customer_lifetime_value,
        round((SUM(unit_price) / count(distinct invoice_id)),2) average_order,
        case
            when country = "Other" then 1
            else 0
        end as sort
    from country_or_other
    group by country
    order by sort, total_sales desc);

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0
